In [1]:
import autokeras as ak
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-07-10 08:30:09.756429: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 데이터 로드
df = pd.read_csv('train.csv') 

# 텍스트 데이터가 500 미만의 길이를 갖도록 자르는 함수
def trim_text(text, max_length=500):
    words = text.split()[:max_length]
    return ' '.join(words)

# 데이터 전처리
df['conversation'] = df['conversation'].apply(trim_text)

# 클래스를 정수로 인코딩
class_dict = {class_name: i for i, class_name in enumerate(df['class'].unique())}
df['class'] = df['class'].map(class_dict)

# 데이터를 학습 및 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(df['conversation'].values, df['class'].values, test_size=0.1, random_state=42)


In [3]:
from datetime import datetime

# AutoKeras text classifier
clf = ak.TextClassifier(
    overwrite=True, 
    max_trials=3,
    multi_label=True,
)

start = datetime.now()
# 모델 학습
clf.fit(X_train, y_train, epochs=10, validation_split=0.15)

print(f'\n\nEnd of train at: {datetime.now()-start}')


Trial 3 Complete [00h 15m 35s]
val_loss: 0.5487021207809448

Best val_loss So Far: 0.25612106919288635
Total elapsed time: 00h 16m 16s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
112/112 [==============================] - 12s 97ms/step - loss: 0.5629 - accuracy: 0.3015
Epoch 2/10
112/112 [==============================] - 0s 4ms/step - loss: 0.4197 - accuracy: 0.6127
Epoch 3/10
112/112 [==============================] - 0s 4ms/step - loss: 0.2404 - accuracy: 0.8222
Epoch 4/10
112/112 [==============================] - 0s 4ms/step - loss: 0.1216 - accuracy: 0.9201
Epoch 5/10
112/112 [==============================] - 1s 4ms/step - loss: 0.0555 - accuracy: 0.9727
Epoch 6/10
112/112 [==============================] - 0s 4ms/step - loss: 0.0245 - accuracy: 0.9910
Epoch 7/10
112/112 [==============================] - 0s 4ms/step - loss: 0.0139 - accuracy: 0.9941
Epoch 8/10
112/112 [==============================] - 0s 4ms/step - loss: 0.0070 - accuracy: 0.9983
Epoch 9/10
112/112 [=====

INFO:tensorflow:Assets written to: ./text_classifier/best_model/assets




End of train at: 0:16:35.482694


In [4]:
# 모델 평가
print(clf.evaluate(X_test, y_test))

13/13 [==============================] - 0s 5ms/step - loss: 0.3964 - accuracy: 0.8177
[0.39644014835357666, 0.8177215456962585]


In [5]:
model = clf.export_model()

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None,)]                 0         
                                                                 
 expand_last_dim (ExpandLas  (None, 1)                 0         
 tDim)                                                           
                                                                 
 text_vectorization (TextVe  (None, 512)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 512, 64)           320064    
                                                                 
 dropout (Dropout)           (None, 512, 64)           0         
                                                                 
 conv1d (Conv1D)             (None, 508, 256)          82176 